In [1]:
import pandas as pd
pd.set_option("display.max_columns", 100)
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
faults = pd.read_csv("../data/J1939Faults.csv")
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")

/var/folders/zw/f8nk99ns29xgk598ct59jt080000gn/T/ipykernel_3264/3872716903.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")
/Users/kd/anaconda3/lib/python3.10/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Data Validation extension is not supported and will be removed
  for idx, row in parser.parse():


In [3]:
diagnostics = pd.pivot(diagnostics, 
                       columns = ['Name'], 
                       values = 'Value', 
                       index = 'FaultId')


In [4]:
# Convert columns to numeric
diagnostics[['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']] = 
diagnostics[['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']].apply(pd.to_numeric, errors = 'coerce')



In [7]:
diagnostics = pd.get_dummies(diagnostics, drop_first=True, columns = ['CruiseControlActive', 'IgnStatus', 'ParkingBrake'])
diagnostics = diagnostics[['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure', 'CruiseControlActive_True', 'IgnStatus_True', 'ParkingBrake_True']]
diagnostics_orig = diagnostics.copy()

In [8]:
predictors = ['AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'ServiceDistance', 'Speed', 'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure']

In [9]:
 scaler = StandardScaler().fit(diagnostics[predictors])

knn_scaled_filled = pd.DataFrame(
    scaler.inverse_transform(KNNImputer().fit_transform(scaler.transform(diagnostics[predictors]))),
    columns = predictors
)

KeyboardInterrupt: 

In [ ]:
diagnostics

In [ ]:
diagnostics.dtypes

In [ ]:
diagnostics.isna().sum()

In [ ]:
diagnostics['ServiceDistance'].count()

In [ ]:
diagnostics.columns.values

In [ ]:
columns  = list(diagnostics.columns)

In [ ]:
predictors = ['']

**There are service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722), so you should remove any records in the vicinity of these locations, as fault codes may be tripped when working on the vehicles.**

In [ ]:
faults = faults.drop(columns = ['ecuSoftwareVersion', 'ecuSerialNumber', 'faultValue'])

In [ ]:
# Changing the data type of the latitude and longitude columns so that I can do a partial search 
# of the values that are near the service locations.
faults[['Latitude', 'Longitude']] = faults[['Latitude', 'Longitude']].astype(str)

In [ ]:
faults[(faults['Latitude'].str.contains(r'36.066|35.58|36.19')) & (faults['Longitude'].str.contains(r'-86.43|-86.44|-83.17'))]

In [ ]:
faults[faults["lat_long"].str.contains(r'36.066'and'-86.43')]

In [ ]:
faults[faults["lat_long"].str.contains(r'36.066') & faults["lat_long"].str.contains(r'-86.43')]


In [ ]:
faults['lat_long']

In [ ]:
faults[faults['Latitude'].str.contains(r'36.19')]

In [ ]:
# removing the rows that had Lat and long values near the service locations.
faults = faults.drop(faults[faults['Latitude'].str.contains(r'36.066|35.58|36.19')].index)

In [ ]:
# creating a column that has the lat long values together.
faults["lat_long"] = list(zip(faults['Latitude'], faults['Longitude']))

In [ ]:
faults.info()

Break down Location Time stamp, into days and hours?

In [ ]:
pd.to_datetime(faults['LocationTimeStamp],
                format = "%Y/%m/%d")